In [1]:
# import the libraries
import numpy as np
import pandas as pd
import re 
from Bio import SeqIO
from collections import defaultdict
import gfapy

In [2]:
# Open the GFA file
file_path = "graph1.gfa"
gfa = gfapy.Gfa.from_file(file_path)

num_segments = len(gfa.segments)

In [3]:
data_graph = []

In [4]:
# Store the name and sequence for each node from the graph
for segment in gfa.segments:
    data_graph.append({"Name": segment.name, "Sequence": segment.sequence})

# Convert the list to a DataFrame
df_graph = pd.DataFrame(data_graph)

# Print the DataFrame
print(df_graph)

       Name                                           Sequence
0      1321  CGTTCCACCGGTTCTTACAGCCTGGTTACTCAGCAGCCGCTGGGTG...
1      1323  GTGCCTTACCACCCAGCGGCTGCTGAGTAACCAGGCTGTAAGAACC...
2      1325  GTGCCTTACCACCCAGCGGCTGCTGAGTAACCAGGCTGTAAGAACC...
3     32989  CTTAATATGAACCATCCAACTTTATGGGGTCAGTCCAGCAGCGCCG...
4       565  GGTTCGGCGGAGCTTACCGCGTCTTTTCGCGGTTAGCGGAGTGTGG...
...     ...                                                ...
2524  37173  GAACAAGGATCTAAGCTGTTTTAAGTTATGGGCAACGCAATGCACT...
2525  24893  TCTTAAGAGAGTGCATTGCGTTGCCCATAACTTAAAACAGCTTAGA...
2526  36779  TTTTCTCTGCAACCGAACCGGCTGTTTGTGTGAAGTGATTCACATC...
2527   6673  CTCGGCCCGACCCGAAGCCTGCAGGGATAAGTCGAAGGGACCGCGC...
2528  37823  CTCGGCCCGACCCGAAGCCTGCAGGGATAAGTCGAAGGGACCGCGC...

[2529 rows x 2 columns]


In [5]:
#Save only one copy of each row to the dataframe
df_graph=df_graph.drop_duplicates()

In [6]:
#GraphAlignerNotEqual.tsv is from another script:GraphAlignerBandageScript.ipynb
df_GraphAligner_Paths= pd.read_csv('GraphAlignerNotEqual.tsv', sep='\t')

In [7]:
df_GraphAligner_Paths

,Unnamed: 0,Query,Path_Bandage,Start Position on Path,End Position on Path,Path Matching,Path_GraphAligner,Equal
0,0,gb|AF047479|+|1295-2087|ARO:3002603|aadA3,7593.0,1687,2477,>7591,7591,no
1,1,gb|AF156486|+|5012-5792|ARO:3002602|aadA2,7593.0,1699,2479,>7591,7591,no
2,3,gb|AF550679.1|-|80976-81768|ARO:3002601|aadA,7591.0,711,1500,>7593,7593,no
3,4,gb|AJ809407|+|118-898|ARO:3002620|aadA23,7591.0,731,1500,>7593,7593,no
4,5,gb|AM040708.1|+|1173-1965|ARO:3004704|aadA8b,7593.0,1687,2477,>7591,7591,no
5,6,gb|AM261837|+|73-865|ARO:3002619|aadA22,7591.0,711,1500,>7593,7593,no
6,10,gb|AY139603|+|106-898|ARO:3002608|aadA8,7593.0,1687,2477,>7591,7591,no
7,11,gb|AY171244|+|46-838|ARO:3002618|aadA21,7591.0,711,1500,>7593,7593,no
8,13,gb|BX664015.1|-|103017-103833|ARO:3002641|APH(...,264523479.0,75,891,<2645<22251,264522251,no
9,14,gb|CP003022|+|336788-337580|ARO:3003197|aadA25,7593.0,1687,2477,>7591,7591,no


In [8]:
# Function to process the Path values
def process_path(path):
    path = path.replace('<', ' ').replace('>', ' ')
    path = ' '.join(path.split('<'))
    return path

In [9]:
df_GraphAligner_Paths['Path Matching']=df_GraphAligner_Paths['Path Matching'].apply(process_path).str.strip()

In [10]:
df_GraphAligner_Paths[['Path1', 'Path2']] = df_GraphAligner_Paths['Path Matching'].str.split(' ', expand=True)

In [11]:
df_GraphAligner_Paths

,Unnamed: 0,Query,Path_Bandage,Start Position on Path,End Position on Path,Path Matching,Path_GraphAligner,Equal,Path1,Path2
0,0,gb|AF047479|+|1295-2087|ARO:3002603|aadA3,7593.0,1687,2477,7591,7591,no,7591,None
1,1,gb|AF156486|+|5012-5792|ARO:3002602|aadA2,7593.0,1699,2479,7591,7591,no,7591,None
2,3,gb|AF550679.1|-|80976-81768|ARO:3002601|aadA,7591.0,711,1500,7593,7593,no,7593,None
3,4,gb|AJ809407|+|118-898|ARO:3002620|aadA23,7591.0,731,1500,7593,7593,no,7593,None
4,5,gb|AM040708.1|+|1173-1965|ARO:3004704|aadA8b,7593.0,1687,2477,7591,7591,no,7591,None
5,6,gb|AM261837|+|73-865|ARO:3002619|aadA22,7591.0,711,1500,7593,7593,no,7593,None
6,10,gb|AY139603|+|106-898|ARO:3002608|aadA8,7593.0,1687,2477,7591,7591,no,7591,None
7,11,gb|AY171244|+|46-838|ARO:3002618|aadA21,7591.0,711,1500,7593,7593,no,7593,None
8,13,gb|BX664015.1|-|103017-103833|ARO:3002641|APH(...,264523479.0,75,891,2645 22251,264522251,no,2645,22251
9,14,gb|CP003022|+|336788-337580|ARO:3003197|aadA25,7593.0,1687,2477,7591,7591,no,7591,None


In [12]:
# Get the rows where Path 2 is not None
rows_not_none = df_GraphAligner_Paths[df_GraphAligner_Paths['Path2'].isna()]

# Print the resulting dataframe
rows_not_none

,Unnamed: 0,Query,Path_Bandage,Start Position on Path,End Position on Path,Path Matching,Path_GraphAligner,Equal,Path1,Path2
0,0,gb|AF047479|+|1295-2087|ARO:3002603|aadA3,7593.0,1687,2477,7591,7591,no,7591,None
1,1,gb|AF156486|+|5012-5792|ARO:3002602|aadA2,7593.0,1699,2479,7591,7591,no,7591,None
2,3,gb|AF550679.1|-|80976-81768|ARO:3002601|aadA,7591.0,711,1500,7593,7593,no,7593,None
3,4,gb|AJ809407|+|118-898|ARO:3002620|aadA23,7591.0,731,1500,7593,7593,no,7593,None
4,5,gb|AM040708.1|+|1173-1965|ARO:3004704|aadA8b,7593.0,1687,2477,7591,7591,no,7591,None
5,6,gb|AM261837|+|73-865|ARO:3002619|aadA22,7591.0,711,1500,7593,7593,no,7593,None
6,10,gb|AY139603|+|106-898|ARO:3002608|aadA8,7593.0,1687,2477,7591,7591,no,7591,None
7,11,gb|AY171244|+|46-838|ARO:3002618|aadA21,7591.0,711,1500,7593,7593,no,7593,None
9,14,gb|CP003022|+|336788-337580|ARO:3003197|aadA25,7593.0,1687,2477,7591,7591,no,7591,None
10,15,gb|DQ393783|+|1799-2591|ARO:3002615|aadA15,7591.0,711,1498,7593,7593,no,7593,None


In [13]:
merged_df= pd.merge(rows_not_none, df_graph, left_on='Path1', right_on='Name')

In [14]:
def extract_sequences(dataframe):
    for index, row in dataframe.iterrows():
        start_pos = row['Start Position on Path']
        end_pos = row['End Position on Path']
        sequence = row['Sequence']
        
        extracted_sequence = sequence[start_pos-1:end_pos]
        dataframe.loc[index, 'Results'] = extracted_sequence
    
    return dataframe

In [16]:
extracted_sequences = extract_sequences(merged_df)
extracted_sequences

,Unnamed: 0,Query,Path_Bandage,Start Position on Path,End Position on Path,Path Matching,Path_GraphAligner,Equal,Path1,Path2,Name,Sequence,Results
0,0,gb|AF047479|+|1295-2087|ARO:3002603|aadA3,7593.0,1687,2477,7591,7591,no,7591,None,7591,TGTTATGGAGCAGCAACGATGTTACGCAGCAGGGCAGTCGCCCTAA...,CATGAGGGTAGCGGTGACCATCGAAATTTCGAACCAACTATCAGAG...
1,1,gb|AF156486|+|5012-5792|ARO:3002602|aadA2,7593.0,1699,2479,7591,7591,no,7591,None,7591,TGTTATGGAGCAGCAACGATGTTACGCAGCAGGGCAGTCGCCCTAA...,GGTGACCATCGAAATTTCGAACCAACTATCAGAGGTGCTAAGCGTC...
2,5,gb|AM040708.1|+|1173-1965|ARO:3004704|aadA8b,7593.0,1687,2477,7591,7591,no,7591,None,7591,TGTTATGGAGCAGCAACGATGTTACGCAGCAGGGCAGTCGCCCTAA...,CATGAGGGTAGCGGTGACCATCGAAATTTCGAACCAACTATCAGAG...
3,10,gb|AY139603|+|106-898|ARO:3002608|aadA8,7593.0,1687,2477,7591,7591,no,7591,None,7591,TGTTATGGAGCAGCAACGATGTTACGCAGCAGGGCAGTCGCCCTAA...,CATGAGGGTAGCGGTGACCATCGAAATTTCGAACCAACTATCAGAG...
4,14,gb|CP003022|+|336788-337580|ARO:3003197|aadA25,7593.0,1687,2477,7591,7591,no,7591,None,7591,TGTTATGGAGCAGCAACGATGTTACGCAGCAGGGCAGTCGCCCTAA...,CATGAGGGTAGCGGTGACCATCGAAATTTCGAACCAACTATCAGAG...
5,18,gb|FJ460181|+|1790-2582|ARO:3002617|aadA17,7593.0,1687,2479,7591,7591,no,7591,None,7591,TGTTATGGAGCAGCAACGATGTTACGCAGCAGGGCAGTCGCCCTAA...,CATGAGGGTAGCGGTGACCATCGAAATTTCGAACCAACTATCAGAG...
6,3,gb|AF550679.1|-|80976-81768|ARO:3002601|aadA,7591.0,711,1500,7593,7593,no,7593,None,7593,TGATAAATGCTTCAATAATATTGAAAAAGGAAGAGTATGAGTATTC...,CATGAGGGAAGCGGTGATCGCCGAAGTATCGACTCAACTATCAGAG...
7,4,gb|AJ809407|+|118-898|ARO:3002620|aadA23,7591.0,731,1500,7593,7593,no,7593,None,7593,TGATAAATGCTTCAATAATATTGAAAAAGGAAGAGTATGAGTATTC...,CCGAAGTATCGACTCAACTATCAGAGGTAGTTGGCGTCATCGAGCG...
8,6,gb|AM261837|+|73-865|ARO:3002619|aadA22,7591.0,711,1500,7593,7593,no,7593,None,7593,TGATAAATGCTTCAATAATATTGAAAAAGGAAGAGTATGAGTATTC...,CATGAGGGAAGCGGTGATCGCCGAAGTATCGACTCAACTATCAGAG...
9,11,gb|AY171244|+|46-838|ARO:3002618|aadA21,7591.0,711,1500,7593,7593,no,7593,None,7593,TGATAAATGCTTCAATAATATTGAAAAAGGAAGAGTATGAGTATTC...,CATGAGGGAAGCGGTGATCGCCGAAGTATCGACTCAACTATCAGAG...


In [17]:
extracted_sequences.to_csv("extracted_sequences.tsv")